In [1]:
import os

# Example notebook name - replace with your actual notebook name
notebook_name = "IAC_dataset_training.ipynb"

# Get the current working directory
current_directory = os.getcwd()

# Construct the full path to the notebook
full_notebook_path = os.path.join(current_directory, notebook_name)

# Set the WANDB_NOTEBOOK_NAME environment variable
os.environ['WANDB_NOTEBOOK_NAME'] = full_notebook_path

# You can print it to verify it's set correctly
print("WANDB_NOTEBOOK_NAME is set to:", os.environ['WANDB_NOTEBOOK_NAME'])

sweep_config = {
    'method': 'bayes',  # Bayesian optimization
    'metric': {
        'name': 'val/seg_loss',
        'goal': 'minimize'   
    },
    'parameters': {
        'epochs': {'value': 20},
        'num_model_checkpoints': {'value': 3},
        'batch': {
            'values': [8, 16, 32, 64]
        },
        'close_mosaic': {
            'values': [0, 5, 10, 15, 20]
        },
        # 'freeze': {
        #     'values': [0, 10, 20, 30]  # Assuming a range; adjust based on model specifics
        # },
        'nbs': {
            'values': [32, 64, 128]
        },
        'mask_ratio': {
            'values': [1, 2, 4, 8]
        },
        'single_cls': {
            'values': [True, False]
        },
        'rect': {
            'values': [True, False]
        },
        'cos_lr': {
            'values': [True, False]
        },
        'overlap_mask': {
            'values': [True, False]
        },
        'optimizer': {
            'values': ['SGD', 'Adam', 'AdamW', 'NAdam', 'RAdam', 'RMSProp']
        },
        'lr0': {
            'min': 0.001,
            'max': 0.01
        },
        'lrf': {
            'min': 0.01,
            'max': 0.1
        },
        'momentum': {
            'min': 0.85,
            'max': 0.95
        },
        'weight_decay': {
            'min': 0.0001,
            'max': 0.001
        },
        'warmup_epochs': {
            'values': [0, 3, 5]
        },
        'warmup_momentum': {
            'min': 0.5,
            'max': 0.95
        },
        'warmup_bias_lr': {
            'min': 0.05,
            'max': 0.2
        },
        'box': {
            'min': 5,
            'max': 10
        },
        'cls': {
            'min': 0.2,
            'max': 0.8
        },
        'dfl': {
            'min': 1,
            'max': 2
        },
        'pose': {
            'min': 10,
            'max': 14
        },
        'kobj': {
            'min': 1,
            'max': 3
        },
        'label_smoothing': {
            'min': 0.0,
            'max': 0.2
        },
        'dropout': {
            'min': 0.0,
            'max': 0.3
        },
        'imgsz': {
            'value': 640
        },
        # 'Car Occurrence Probability': {
        #     'min': 0.1,
        #     'max': 0.9
        # }
    }
}


WANDB_NOTEBOOK_NAME is set to: /home/chris-lai/yolov8/yolov8/IAC_dataset_training.ipynb


In [ ]:
import wandb
from ultralytics import YOLO
import os
import json
import datetime

# Path to the checkpoint file
WANDB_PROJECT_NAME = "IAC Yolov8 Deployment Testing1"
checkpoint_path = "sweep_checkpoint.json"

def save_checkpoint(config):
    # Convert wandb.config to a regular dictionary
    config_dict = {k: v for k, v in config.items()}
    with open(checkpoint_path, 'w') as f:
        json.dump(config_dict, f)

def load_checkpoint():
    if os.path.exists(checkpoint_path) and os.path.getsize(checkpoint_path) > 0:
        with open(checkpoint_path) as f:
            return json.load(f)
    return None


def train():
    # Check if resume is requested
    checkpoint_config = load_checkpoint()
    
    if checkpoint_config:
        resume_config = checkpoint_config
    else:
        resume_config = None

    # Initialize a new WandB run
    with wandb.init(resume=resume_config, entity=) as run:
        print(run)
        config = wandb.config
        
        # Checkpoint current hyperparameters
        save_checkpoint(config)

        # Generate a timestamp
        timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

        # Include the timestamp in the run name for uniqueness
        name = f"batch_{config.batch}_lr0_{config.lr0}_momentum_{config.momentum}_imgsz_{config.imgsz}_{timestamp}"
        model = YOLO("yolov8n-seg.pt")
        
        model.train(
            project=WANDB_PROJECT_NAME,
            name=name,  # Dynamically generated unique name including the timestamp
            data="../../datasets/testing_dataset/data.yaml",
            epochs=config.epochs,
            imgsz=config.imgsz,
            batch=config.batch,
            lr0=config.lr0,
            lrf=config.lrf,
            momentum=config.momentum,
            weight_decay=config.weight_decay,
            warmup_epochs=config.warmup_epochs,
            warmup_momentum=config.warmup_momentum,
            warmup_bias_lr=config.warmup_bias_lr,
            box=config.box,
            cls=config.cls,
            dfl=config.dfl,
            pose=config.pose,
            kobj=config.kobj,
            save=True,
            save_period=config.epochs // config.num_model_checkpoints,
            # Additional parameters mapped from config to model.train() arguments
            single_cls=config.single_cls,
            rect=config.rect,
            cos_lr=config.cos_lr,
            optimizer=config.optimizer,
            label_smoothing=config.label_smoothing,
            dropout=config.dropout,
            close_mosaic=config.close_mosaic,
            # freeze=config.freeze,
            nbs=config.nbs,
            mask_ratio=config.mask_ratio,
            overlap_mask=config.overlap_mask,
            
            # Note: Some parameters like 'freeze', 'nbs', 'mask_ratio' might require
            # pre-training model adjustments or are not directly used in model.train()
        )

        # Optionally, delete checkpoint on successful completion
        if os.path.exists(checkpoint_path):
            os.remove(checkpoint_path)

if __name__ == "__main__":
    # Create the sweep
    sweep_id = wandb.sweep(sweep=sweep_config, project=WANDB_PROJECT_NAME)
    wandb.agent(sweep_id, train)

In [7]:
wandb.finish()